# Analysis Purpose
>### Caculating blue pixel number to predict image whether is unfilled deed

## Table of Contents
* [1. Import Packages](#1.-Import-Packages)
* [2. Set Direction and Find the File Path](#2.-Set-Direction-and-Find-the-File-Path)
* [3. Model Build by Image RGB Threshold](#3.-Model-Build-by-Image-RGB-Threshold)
    * [3.1 Save Model Result](#3.1-Save-Model-Result)
* [4. check no blue pixel image result](#4.-check-no-blue-pixel-image-result)

## 1. Import Packages   
[Go back to the Table of Contents](#Table-of-Contents)

In [1]:
# import the necessary packages
import numpy as np
import argparse
import cv2
import matplotlib.pyplot as plt
import os
from glob import glob
import pandas as pd
from tqdm import tqdm
import random

## 2. Set Direction and Find the File Path
[Go back to the Table of Contents](#Table-of-Contents)

In [2]:
deed_name = ['AA', 'AD', 'AG', 'BA', 'BD', 'BG', 'EA', 'ED', 'EG', 'HA', 'HD',
             'HG', 'LA', 'LG', 'PA', 'PG', 'QA', 'QG', 'RA', 'RG', 'UA', 'UD', 'UG', ]

In [3]:
ok_file_name = ['P2_OK_P4', 'P3_OK_P4']

In [4]:
ok_deed_dir = "//Srvappweb-t/f$/{}/{}/".format(deed_name[1], ok_file_name[1])

In [5]:
ok_deed_dir_li = []
for i in range(len(deed_name)):
    for t in range(len(ok_file_name)):
        ok_deed_dir = "//Srvappweb-t/f$/{}/{}/".format(deed_name[i], ok_file_name[t])
        ok_deed_dir_li.append(ok_deed_dir)

In [6]:
img_path_list = []
for ok_deed_dir in ok_deed_dir_li:
    for file_path in glob('{}/*.jpg'.format(ok_deed_dir)):
        img_path_list.append(file_path)

In [7]:
img_path_list[0]

'//Srvappweb-t/f$/AA/P2_OK_P4\\AB002-AA0769076.jpg'

In [8]:
print(f'Number of ok deed: {len(img_path_list)}')

Number of ok deed: 69912


## 3. Model Build by Image RGB Threshold
[Go back to the Table of Contents](#Table-of-Contents)

In [9]:
# 查看類別資料夾裡的資料
result = []
for i in tqdm(range(len(img_path_list))):
    if i == 5000:
        break
    # 讀取圖檔
    img_path = cv2.imread(img_path_list[i])
    img = cv2.cvtColor(img_path, cv2.COLOR_BGR2RGB)



    img = cv2.resize(img,(675, 900))


    # 裁切區域的 x 與 y 座標（左上角）
    x = 100
    y = 30

    # 裁切區域的長度與寬度
    w = 525
    h = 770

    # 裁切圖片
    cut_img = img[y:y+h, x:x+w]

    color_dict = {'blue_0': [(117, 0, 0),(255, 255, 108)],
                  'blue_1': [(121, 0, 0),(255, 115, 152)]
                  }


    blue_0_min = color_dict['blue_0'][0]
    blue_0_max = color_dict['blue_0'][1]
    blue_1_min = color_dict['blue_1'][0]
    blue_1_max = color_dict['blue_1'][1]

    # return 一個值為 0 或 255的矩陣
    find_blue_pixel_0 = cv2.inRange(cut_img, blue_0_min, blue_0_max)
    find_blue_pixel_1 = cv2.inRange(cut_img, blue_1_min, blue_1_max)
    find_blue_pixel = find_blue_pixel_0+find_blue_pixel_1
    
    blue_color = len(find_blue_pixel[(find_blue_pixel >= 240)])/(find_blue_pixel.shape[0]*find_blue_pixel.shape[1])
    
    
    result.append(blue_color)

  7%|▋         | 5000/69912 [14:38<3:10:03,  5.69it/s] 


## 3.1 Save Model Result
[Go back to the Table of Contents](#Table-of-Contents)

In [10]:
img_path_df = pd.DataFrame(img_path_list, columns = ['path'])

In [11]:
result = pd.DataFrame(result, columns = ['blue_color'])

data_result_df = pd.concat([img_path_df, result], axis = 1)
data_result_df.to_csv('./data_result_df.csv', index = False)

In [12]:
data_result_df.head()

,path,blue_color
0,//Srvappweb-t/f$/AA/P2_OK_P4\AB002-AA0769076.jpg,0.000139
1,//Srvappweb-t/f$/AA/P2_OK_P4\AB002-AA0769077.jpg,0.002778
2,//Srvappweb-t/f$/AA/P2_OK_P4\AB002-AA0769078.jpg,0.000042
3,//Srvappweb-t/f$/AA/P2_OK_P4\AB002-AA0769097.jpg,0.019466
4,//Srvappweb-t/f$/AA/P2_OK_P4\AB002-AA0816252.jpg,0.010964


In [13]:
unfilled_deed = data_result_df[data_result_df.blue_color<0.0001]

In [14]:
print(f'unfilled_deed: {len(unfilled_deed)}')

unfilled_deed: 184


## 4. check no blue pixel image result
[Go back to the Table of Contents](#Table-of-Contents)

In [ ]:
image = random.choice(unfilled_deed['path'].tolist())
img = cv2.imread(image)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.rcParams["figure.figsize"] = (20,20)
plt.imshow(img)